# Tratamento do dataset df3

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('Prata2') \
  .getOrCreate()

In [5]:
from pyspark.sql.functions import count_distinct, col, when, sum, regexp_extract

In [6]:
df = spark.read.csv("dados\df3.csv", sep =";", header=True)

In [7]:
df.show()

+--------------------+-------------------+--------------------+
|      Característica|Aval.Característica|                 URL|
+--------------------+-------------------+--------------------+
|     Custo-benefício| Avaliação 4.6 de 5|https://www.merca...|
|            Absorção| Avaliação 4.7 de 5|https://www.merca...|
|Qualidade dos mat...| Avaliação 4.6 de 5|https://www.merca...|
|     Custo-benefício| Avaliação 4.6 de 5|https://www.merca...|
|     Sabor agradável| Avaliação 4.7 de 5|https://www.merca...|
|            Benéfico| Avaliação 4.7 de 5|https://www.merca...|
|    Fácil de digerir| Avaliação 4.7 de 5|https://www.merca...|
|     Custo-benefício| Avaliação 4.8 de 5|https://www.merca...|
|     Sabor agradável| Avaliação 4.8 de 5|https://www.merca...|
|    Fácil de digerir| Avaliação 4.8 de 5|https://www.merca...|
|            Benéfico| Avaliação 4.6 de 5|https://www.merca...|
|     Custo-benefício| Avaliação 4.6 de 5|https://produto.m...|
|Qualidade dos mat...|   Avaliação 5 de 

In [8]:
df.count()

4256

In [9]:
df.printSchema()

root
 |-- Característica: string (nullable = true)
 |-- Aval.Característica: string (nullable = true)
 |-- URL: string (nullable = true)



In [10]:
df = df.withColumnRenamed("Aval.Característica", "Aval_Caracteristica").withColumnRenamed("Característica", "Caracteristica")

In [11]:
df.select("Caracteristica").distinct().withColumnRenamed("Caracteristica", "Caracteristiscas_Unicas").show(truncate = False)
df.select(count_distinct(col("Caracteristica")).alias("Caracteristicas_Distintas")).show()

+--------------------------------+
|Caracteristiscas_Unicas         |
+--------------------------------+
|Fragrância                      |
|Não encolhe na lavagem          |
|Sabor agradável                 |
|Benéfico                        |
|Confortável                     |
|Custo-benefício                 |
|Relación precio-calidad         |
|É quente                        |
|Resistente às condições externas|
|Calidad de los materiales       |
|Resistência                     |
|Qualidade do tecido             |
|Fácil de digerir                |
|Praticidade                     |
|Absorção                        |
|Durabilidade                    |
|Qualidade dos materiais         |
+--------------------------------+

+-------------------------+
|Caracteristicas_Distintas|
+-------------------------+
|                       17|
+-------------------------+



In [12]:
df1 = df.withColumn("Caracteristica", when(col("Caracteristica") == "Relación precio-calidad", "Relação preço-qualidade").otherwise(col("Caracteristica"))). \
withColumn("Caracteristica", when(col("Caracteristica") == "Calidad de los materiales", "Qualidade dos materiais").otherwise(col("Caracteristica")))
df1.select("Caracteristica").distinct().show(truncate = False)
df1.select(count_distinct(col("Caracteristica")).alias("Caracteristicas_Distintas")).show()

+--------------------------------+
|Caracteristica                  |
+--------------------------------+
|Fragrância                      |
|Não encolhe na lavagem          |
|Sabor agradável                 |
|Benéfico                        |
|Confortável                     |
|Custo-benefício                 |
|É quente                        |
|Resistente às condições externas|
|Resistência                     |
|Qualidade do tecido             |
|Fácil de digerir                |
|Praticidade                     |
|Absorção                        |
|Relação preço-qualidade         |
|Durabilidade                    |
|Qualidade dos materiais         |
+--------------------------------+

+-------------------------+
|Caracteristicas_Distintas|
+-------------------------+
|                       16|
+-------------------------+



In [13]:
nulos = df.select([sum(col("Aval_Caracteristica").isNull().cast("int")).alias("Quant_Nulos"), \
                                      sum(col("Caracteristica").isNull().cast("int")).alias("Quant_Caracteristica"), \
                                       sum(col("URL").isNull().cast("int")).alias("Quant_URL")])
nulos.show()

+-----------+--------------------+---------+
|Quant_Nulos|Quant_Caracteristica|Quant_URL|
+-----------+--------------------+---------+
|          0|                   0|        0|
+-----------+--------------------+---------+



In [14]:
df2 = df.withColumn("Aval_Caracteristica", regexp_extract("Aval_Caracteristica", r"(\d+(\.\d+)?)", 1))

In [15]:
df2.show()

+--------------------+-------------------+--------------------+
|      Caracteristica|Aval_Caracteristica|                 URL|
+--------------------+-------------------+--------------------+
|     Custo-benefício|                4.6|https://www.merca...|
|            Absorção|                4.7|https://www.merca...|
|Qualidade dos mat...|                4.6|https://www.merca...|
|     Custo-benefício|                4.6|https://www.merca...|
|     Sabor agradável|                4.7|https://www.merca...|
|            Benéfico|                4.7|https://www.merca...|
|    Fácil de digerir|                4.7|https://www.merca...|
|     Custo-benefício|                4.8|https://www.merca...|
|     Sabor agradável|                4.8|https://www.merca...|
|    Fácil de digerir|                4.8|https://www.merca...|
|            Benéfico|                4.6|https://www.merca...|
|     Custo-benefício|                4.6|https://produto.m...|
|Qualidade dos mat...|                  

In [16]:
df3 = df2.withColumn("Aval_Caracteristica",col("Aval_Caracteristica").cast("double"))

In [17]:
df3.printSchema()

root
 |-- Caracteristica: string (nullable = true)
 |-- Aval_Caracteristica: double (nullable = true)
 |-- URL: string (nullable = true)



In [18]:
df3.show()

+--------------------+-------------------+--------------------+
|      Caracteristica|Aval_Caracteristica|                 URL|
+--------------------+-------------------+--------------------+
|     Custo-benefício|                4.6|https://www.merca...|
|            Absorção|                4.7|https://www.merca...|
|Qualidade dos mat...|                4.6|https://www.merca...|
|     Custo-benefício|                4.6|https://www.merca...|
|     Sabor agradável|                4.7|https://www.merca...|
|            Benéfico|                4.7|https://www.merca...|
|    Fácil de digerir|                4.7|https://www.merca...|
|     Custo-benefício|                4.8|https://www.merca...|
|     Sabor agradável|                4.8|https://www.merca...|
|    Fácil de digerir|                4.8|https://www.merca...|
|            Benéfico|                4.6|https://www.merca...|
|     Custo-benefício|                4.6|https://produto.m...|
|Qualidade dos mat...|                5.

In [19]:
df3.count()

4256

In [20]:
id_urls = spark.read.csv("dados\IDs_URLs.csv", sep = ";", header = True)
df3_tratado = id_urls.join(df3, on="URL", how="inner")
df3_tratado = df3_tratado.drop("URL")
df3_tratado.show()

+----+--------------------+-------------------+
|  ID|      Caracteristica|Aval_Caracteristica|
+----+--------------------+-------------------+
|1419|     Custo-benefício|                4.6|
|1419|            Absorção|                4.7|
|1419|Qualidade dos mat...|                4.6|
|1500|     Custo-benefício|                4.6|
|1500|     Sabor agradável|                4.7|
|1500|            Benéfico|                4.7|
|1500|    Fácil de digerir|                4.7|
| 631|     Custo-benefício|                4.8|
| 631|     Sabor agradável|                4.8|
| 631|    Fácil de digerir|                4.8|
| 631|            Benéfico|                4.6|
|1703|     Custo-benefício|                4.6|
|1703|Qualidade dos mat...|                5.0|
|1703|        Durabilidade|                5.0|
|2059|     Custo-benefício|                4.2|
|2059|        Durabilidade|                4.7|
|2059|         Resistência|                4.7|
|2005|            Absorção|             

In [ ]:
df3_tratado.write.parquet("dados\df3_tratado.parquet")

In [21]:
df3_tratado.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Caracteristica: string (nullable = true)
 |-- Aval_Caracteristica: double (nullable = true)

